# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst, as they have the expertise in statistical modeling and machine learning.

Action: Delegate work to co-worker
Action Input: 
{
    "coworker": "Data Analyst",
    "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning",
    "context": "The task requires expertise in statistical modeling and machine learning to identify trends and predict market movements for AAPL."
}


> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL and analyze it using statistical modeling and machine learning to predict market movements. I s

Final Answer: 
Based on the real-time market data for AAPL from Yahoo Finance, the current stock price for Apple Inc. (AAPL) is $195.67 with a slight decrease of 0.10%. The market cap is at $3.00 trillion, and the company's key financial highlights show a profit margin of 26.31%, return on assets of 22.07%, and a revenue of $381.62 billion. Additionally, the trailing P/E ratio is 30.46, and the forward P/E ratio is 26.67. This information can be further analyzed using statistical modeling and machine learning to predict market movements.

> Finished chain.
 

Based on the real-time market data for AAPL from Yahoo Finance, the current stock price for Apple Inc. (AAPL) is $195.67 with a slight decrease of 0.10%. The market cap is at $3.00 trillion, and the company's key financial highlights show a profit margin of 26.31%, return on assets of 22.07%, and a revenue of $381.62 billion. Additionally, the trailing P/E ratio is 30.46, and the forward P/E ratio is 26.67. This information can be

Final Answer: 

After conducting thorough research and analysis, I have identified several potential day trading strategies for AAPL that align with the user's risk tolerance level. The strategies include the trend moving strategy, simple day trading strategies tailored for AAPL stock, AAPL option strategies, and day trading AAPL options. These strategies aim to optimize trading performance while considering the potential uptrend in AAPL stock price and market volatility. It is crucial to closely monitor market trends and news related to AAPL to make well-informed trading decisions.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: After conducting thorough research and analysis, I have identified several potential day trading strategies for AAPL that align with the user's risk tolerance level. The strategies include the trend moving strategy, simple day trading strategies tailored for AAPL stock, AAPL option strategies, and day trading AAPL options. These strategies aim to opti

Final Answer: 
After conducting a search on approved trading strategies for AAPL, I found a variety of resources that provide insights into different trading strategies for Apple stock. Some of the strategies mentioned include options trading, earnings volatility trading, and considerations for building a trading strategy specific to AAPL. 

To evaluate the risks associated with these strategies, it is important to analyze the potential gains and losses, market conditions, and historical data related to Apple's stock performance. Additionally, it would be beneficial to monitor the overall market trends and news that could impact AAPL's stock price.

In order to propose mitigation strategies for the risks involved, it would be advisable to diversify the trading portfolio, set stop-loss orders, and stay informed about any upcoming events or announcements that could affect Apple's stock. By carefully monitoring the market data in real-time and utilizing statistical modeling and machine le

Thought: I now know the final answer

Final Answer:
Title: Comprehensive Risk Analysis and Mitigation Recommendations for AAPL Trading Strategies

After conducting thorough research and gathering insights from various sources, I have compiled a comprehensive risk analysis and mitigation recommendations for the proposed trading strategies for AAPL.

1. Potential Risks:
- Execution Risks: Risks related to the effective execution of corporate activities and strategies, which can impact stock performance.
- Market Risks: Volatility in the stock market and changes in market conditions that can affect AAPL's stock price.
- Competition Risks: Risks associated with competition in the tech industry, which can impact Apple's market share and profitability.
- Supply Chain Risks: Potential disruptions in the global supply chain that could affect Apple's production and sales.
- Antitrust Risks: Legal risks related to antitrust issues and regulatory challenges that can impact Apple's business operat

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

Title: Comprehensive Risk Analysis and Mitigation Recommendations for AAPL Trading Strategies

After conducting thorough research and gathering insights from various sources, I have compiled a comprehensive risk analysis and mitigation recommendations for the proposed trading strategies for AAPL.

1. Potential Risks:
- Execution Risks: Risks related to the effective execution of corporate activities and strategies, which can impact stock performance.
- Market Risks: Volatility in the stock market and changes in market conditions that can affect AAPL's stock price.
- Competition Risks: Risks associated with competition in the tech industry, which can impact Apple's market share and profitability.
- Supply Chain Risks: Potential disruptions in the global supply chain that could affect Apple's production and sales.
- Antitrust Risks: Legal risks related to antitrust issues and regulatory challenges that can impact Apple's business operations.
- Geopolitical Risks: Risks arising from tensions between the U.S. and China, which can affect Apple's international operations and sales.

2. Mitigation Strategies:
- Diversification: Diversify the trading portfolio to spread out risks and minimize potential losses.
- Risk Management: Implement risk management strategies such as setting stop-loss orders to limit losses.
- Stay Informed: Stay updated on market trends, news, and events that could impact AAPL's stock price.
- Monitor Competition: Keep an eye on competitors and industry trends to anticipate potential risks.
- Regulatory Compliance: Ensure compliance with antitrust laws and regulations to mitigate legal risks.
- Global Relations: Monitor geopolitical developments and maintain good relations with key markets to mitigate international risks.

By addressing these potential risks with the recommended mitigation strategies, we can optimize the trading strategies for AAPL and make informed decisions to maximize returns while minimizing risks.